In [1]:
import xml.etree.ElementTree as ET 

xmlFileName = "export.xml"
tree = ET.parse(xmlFileName)

root = tree.getroot() 

ColumnNames = ["startDate", "endDate", "creationDate", "sourceVersion", "value"]
parquetName = "appleWatchHypnoDf.parquet.gzip"

rType = "HKCategoryTypeIdentifierSleepAnalysis"
sourceName = "Abhik\'s Apple Watch"


some example data

 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:13:59 -0700" endDate="2022-10-01 01:29:59 -0700" value="HKCategoryValueSleepAnalysisAsleepDeep">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>
 </Record>
 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:29:59 -0700" endDate="2022-10-01 01:30:29 -0700" value="HKCategoryValueSleepAnalysisAwake">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>
 </Record>
 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:30:29 -0700" endDate="2022-10-01 01:43:29 -0700" value="HKCategoryValueSleepAnalysisAsleepCore">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>
 </Record>
 <Record type="HKCategoryTypeIdentifierSleepAnalysis" sourceName="Abhik’s Apple Watch" sourceVersion="9.0" creationDate="2022-10-01 09:03:33 -0700" startDate="2022-10-01 01:43:29 -0700" endDate="2022-10-01 01:50:59 -0700" value="HKCategoryValueSleepAnalysisAsleepREM">
  <MetadataEntry key="HKTimeZone" value="America/Phoenix"/>

In [2]:
print(len(root.findall("./Record")))

1182288


In [3]:
import pandas as pd

numRecords = 0
listRecords = []

sourceName = "Abhik’s Apple Watch"

val = 8

# for every element tagged Record
for r in root.findall("./Record"):
    # the data is in the attributes 
    record = r.attrib
    #if record["type"] == rType:
    #    print(record)
    if (record["type"] == rType and record["sourceName"] == sourceName): 
        numRecords += 1
        if numRecords % 100_000 == 0: print(numRecords)

        # the columns that I'm looking for are
        # sleep stage, start time, end time, source version
        # print(record["value"])
        val = -2
        if record["value"] == "HKCategoryValueSleepAnalysisAwake":
            val = 0
        elif record["value"] == "HKCategoryValueSleepAnalysisAsleepCore":
            val = 1
        elif record["value"] == "HKCategoryValueSleepAnalysisAsleepDeep":
            val = 2
        elif record["value"] == "HKCategoryValueSleepAnalysisAsleepREM":
            val = 3

        if val != -2:
            row = [pd.to_datetime(record["startDate"]),
                   pd.to_datetime(record["endDate"]),
                   pd.to_datetime(record["creationDate"]), 
                   record["sourceVersion"],
                   val]
        
            listRecords.append(row)



print(len(listRecords))
print(numRecords)

17045
22725


In [4]:
set([x[4] for x in listRecords])

{0, 1, 2, 3}

In [5]:
from pytz import FixedOffset

tz = FixedOffset(-420)

# add in the no data segments
filledRecords = []
# for every row except the last
for ri in range(len(listRecords)-1):
    filledRecords.append(listRecords[ri])
    # if there is a gap between this rows end date and the next rows start date
    if listRecords[ri][1] < listRecords[ri+1][0]:
        filledRecords.append([
            listRecords[ri][1],
            listRecords[ri+1][0],
            pd.Timestamp.now(tz=tz),
            "synth",
            -1
        ])
print(len(filledRecords))

17571


In [6]:
print(len(filledRecords))

17571


In [7]:
filledRecords[:100]

[[Timestamp('2022-10-01 00:42:59-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 01:13:59-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 09:03:33-0700', tz='pytz.FixedOffset(-420)'),
  '9.0',
  1],
 [Timestamp('2022-10-01 01:13:59-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 01:29:59-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 09:03:33-0700', tz='pytz.FixedOffset(-420)'),
  '9.0',
  2],
 [Timestamp('2022-10-01 01:29:59-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 01:30:29-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 09:03:33-0700', tz='pytz.FixedOffset(-420)'),
  '9.0',
  0],
 [Timestamp('2022-10-01 01:30:29-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 01:43:29-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 09:03:33-0700', tz='pytz.FixedOffset(-420)'),
  '9.0',
  1],
 [Timestamp('2022-10-01 01:43:29-0700', tz='pytz.FixedOffset(-420)'),
  Timestamp('2022-10-01 01:50:

In [8]:
ColumnNames = ["startDate", "endDate", "creationDate", "sourceVersion", "value"]

AppleWatchHypnoDf = pd.DataFrame(columns=ColumnNames, data=filledRecords)

AppleWatchHypnoDf = AppleWatchHypnoDf.set_index("startDate")

In [9]:
AppleWatchHypnoDf

,endDate,creationDate,sourceVersion,value
startDate,,,,
2022-10-01 00:42:59-07:00,2022-10-01 01:13:59-07:00,2022-10-01 09:03:33-07:00,9.0,1
2022-10-01 01:13:59-07:00,2022-10-01 01:29:59-07:00,2022-10-01 09:03:33-07:00,9.0,2
2022-10-01 01:29:59-07:00,2022-10-01 01:30:29-07:00,2022-10-01 09:03:33-07:00,9.0,0
2022-10-01 01:30:29-07:00,2022-10-01 01:43:29-07:00,2022-10-01 09:03:33-07:00,9.0,1
2022-10-01 01:43:29-07:00,2022-10-01 01:50:59-07:00,2022-10-01 09:03:33-07:00,9.0,3
...,...,...,...,...
2024-07-16 07:14:28-07:00,2024-07-16 07:14:58-07:00,2024-07-16 08:13:26-07:00,10.4,0
2024-07-16 07:14:58-07:00,2024-07-16 07:34:58-07:00,2024-07-16 08:13:26-07:00,10.4,1
2024-07-16 07:34:58-07:00,2024-07-16 07:36:28-07:00,2024-07-16 08:13:26-07:00,10.4,0


In [10]:
AppleWatchHypnoDf.dtypes

endDate          datetime64[ns, pytz.FixedOffset(-420)]
creationDate     datetime64[ns, pytz.FixedOffset(-420)]
sourceVersion                                    object
value                                             int64
dtype: object

In [11]:
AppleWatchHypnoDf.to_parquet(parquetName,
              compression='gzip') 